# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [31]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
spark = SparkSession.builder.appName("Data Wrangling Coding Quiz").getOrCreate()
# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
sparkify_log = spark.read.json(path)
# 4) write code to answer the quiz questions 

# Question 1

Which page did user id "" (empty string) NOT visit?

In [2]:
# TODO: write your code to answer question 1
pages_empty_user =sparkify_log.select("page").where(sparkify_log.userId =="").dropDuplicates().collect()
pages_all_users = sparkify_log.select("page").dropDuplicates().collect()
print(pages_all_users)
print(pages_empty_user)
print(set(pages_all_users) - set(pages_empty_user))

[Row(page='Submit Downgrade'), Row(page='Home'), Row(page='Downgrade'), Row(page='Logout'), Row(page='Save Settings'), Row(page='About'), Row(page='Settings'), Row(page='Login'), Row(page='NextSong'), Row(page='Help'), Row(page='Upgrade'), Row(page='Error'), Row(page='Submit Upgrade')]
[Row(page='Home'), Row(page='About'), Row(page='Login'), Row(page='Help')]
{Row(page='NextSong'), Row(page='Logout'), Row(page='Settings'), Row(page='Downgrade'), Row(page='Submit Upgrade'), Row(page='Submit Downgrade'), Row(page='Error'), Row(page='Save Settings'), Row(page='Upgrade')}


In [3]:
print(type(pages_empty_user))
print(len(pages_all_users), len(pages_empty_user))

<class 'list'>
13 4


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [4]:
# TODO: use this space to explore the behavior of the user with an empty string


# Question 3

How many female users do we have in the data set?

In [5]:
sparkify_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
sparkify_log.select("gender").dropDuplicates().collect()

[Row(gender='F'), Row(gender=None), Row(gender='M')]

In [7]:
# TODO: write your code to answer question 3
sparkify_log.select("userId").filter( sparkify_log["gender"] == "F").dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [8]:
type(sparkify_log)

pyspark.sql.dataframe.DataFrame

In [9]:
from pyspark.sql.functions import countDistinct, avg, count
import pyspark.sql.functions as SFunc

In [10]:
# TODO: write your code to answer question 4
sparkify_log.groupBy("artist").agg( countDistinct("song").alias("num_songs") ).sort("num_songs", ascending = False).collect()

[Row(artist='The Black Keys', num_songs=25),
 Row(artist='Coldplay', num_songs=24),
 Row(artist='Jack Johnson', num_songs=20),
 Row(artist='Kings Of Leon', num_songs=20),
 Row(artist='Muse', num_songs=19),
 Row(artist='Nickelback', num_songs=18),
 Row(artist='Daft Punk', num_songs=18),
 Row(artist='Lily Allen', num_songs=17),
 Row(artist='Red Hot Chili Peppers', num_songs=17),
 Row(artist='Radiohead', num_songs=17),
 Row(artist='John Mayer', num_songs=14),
 Row(artist='Gorillaz', num_songs=14),
 Row(artist='Metallica', num_songs=14),
 Row(artist='Weezer', num_songs=14),
 Row(artist='Creedence Clearwater Revival', num_songs=14),
 Row(artist="Guns N' Roses", num_songs=14),
 Row(artist='Kanye West', num_songs=13),
 Row(artist='Rammstein', num_songs=13),
 Row(artist='The All-American Rejects', num_songs=13),
 Row(artist='Foo Fighters', num_songs=13),
 Row(artist='Lifehouse', num_songs=12),
 Row(artist='The Killers', num_songs=12),
 Row(artist='Arctic Monkeys', num_songs=12),
 Row(artist='T

In [11]:
type(sparkify_log.groupBy("artist").agg( countDistinct("song").alias("num_songs") ))

pyspark.sql.dataframe.DataFrame

In [12]:
 sparkify_log.groupBy("artist").agg( countDistinct("song").alias("num_songs") ).select(SFunc.max("num_songs")).collect()

[Row(max(num_songs)=25)]

### Number of distinct songs listend by artist

In [13]:
group_by_artist = sparkify_log.groupBy("artist").agg( countDistinct("song").alias("num_songs") )
max_num_songs = group_by_artist.select(SFunc.max("num_songs")).collect()[0][0]
print(max_num_songs)
group_by_artist.filter( group_by_artist["num_songs"] == max_num_songs).collect()[0]

25


Row(artist='The Black Keys', num_songs=25)

In [14]:
group_by_artist = sparkify_log.groupBy("artist").agg( SFunc.countDistinct("song").alias("num_songs") )
max_num_songs = group_by_artist.select(SFunc.max("num_songs")).collect()[0][0]
print(max_num_songs)

25


### Total number of songplays per artist

In [15]:
songplays_by_artist = sparkify_log.groupBy("artist").agg( SFunc.count("song").alias("num_songlays") )
max_songplays = songplays_by_artist.select(SFunc.max("num_songlays")).collect()[0][0]
print(max_songplays)

83


# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [34]:
from pyspark.sql import Window
from pyspark.sql.functions import desc, asc
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [35]:
l_pages = sparkify_log.select("page").dropDuplicates().collect()
l_pages

[Row(page='Submit Downgrade'),
 Row(page='Home'),
 Row(page='Downgrade'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='About'),
 Row(page='Settings'),
 Row(page='Login'),
 Row(page='NextSong'),
 Row(page='Help'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

In [45]:
for page in l_pages :
    #print(page[0], type(page[0]))
    l_songs = sparkify_log.filter(sparkify_log["page"] == page[0] ).select("song").dropDuplicates().collect()
    print(page[0], len(l_songs) )
    if len(l_songs) < 10 :
        print("\t", l_songs)

Submit Downgrade 1
	 [Row(song=None)]
Home 1
	 [Row(song=None)]
Downgrade 1
	 [Row(song=None)]
Logout 1
	 [Row(song=None)]
Save Settings 1
	 [Row(song=None)]
About 1
	 [Row(song=None)]
Settings 1
	 [Row(song=None)]
Login 1
	 [Row(song=None)]
NextSong 6125
Help 1
	 [Row(song=None)]
Upgrade 1
	 [Row(song=None)]
Error 1
	 [Row(song=None)]
Submit Upgrade 1
	 [Row(song=None)]


### Create windows on users, ordered by timestamp, and ranging from the user first activity to the current activity

In [57]:
window_users = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

### New column with 1 if Home visited, 0 else

In [58]:
flag_home_func = udf(lambda x : 1 if x == "Home" else  0, IntegerType())

In [59]:
sparkify_log_2 = sparkify_log.filter( (sparkify_log["page"] == "Home" )
                                     | (sparkify_log["page"] == "NextSong" ) )\
                            .withColumn("home", flag_home_func("page"))

In [60]:
sparkify_log_2.head()

Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', home=0)

### Number of times Home has been visited

In [61]:
sparkify_log_3 = sparkify_log_2.withColumn("num_home", SFunc.sum("home").over(window_users))

In [62]:
sparkify_log_3.select(["userId", "num_home"]).collect()

[Row(userId='1436', num_home=0),
 Row(userId='1436', num_home=0),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userId='2162', num_home=0),
 Row(userI

### Filter out num_home = 0 (when Home has not yet been visited)

In [63]:
sparkify_log_4 = sparkify_log_3.filter(sparkify_log_3["num_home"] != 0)
sparkify_log_4.select(["userId", "num_home"]).collect()

[Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2088', num_home=1),
 Row(userId='2162', num_home=1),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userId='2162', num_home=2),
 Row(userI

In [64]:
sparkify_log_5 = sparkify_log_3.groupBy(["userId", "num_home"]).agg( SFunc.count("userId").alias("count_songs")).select(["userId", "num_home", "count_songs"])

In [65]:
sparkify_log_5.select(SFunc.avg("count_songs")).collect()

[Row(avg(count_songs)=5.030801911842804)]

In [66]:
sparkify_log_3.filter((sparkify_log_3.page == 'NextSong')) \
    .groupBy('userID', 'num_home') \
    .agg({'num_home':'count'}) \
    .agg({'count(num_home)':'avg'}).show()

+--------------------+
|avg(count(num_home))|
+--------------------+
|   6.898347107438017|
+--------------------+



In [69]:
sparkify_log_3.groupBy(["userId", "num_home"]) \
    .agg({'num_home':'count'}) \
    .agg({'count(num_home)':'avg'}).show()

+--------------------+
|avg(count(num_home))|
+--------------------+
|   5.030801911842804|
+--------------------+

